In [1]:
#Sistema de recomendación con productos Financieros:
# Creado por Felipe Valdez, Omar Vergara, Juan Sebastian, Juan Sebastian Salinas y Gonzalo Moreno
# Fecha 5 de agosto de 2017


# Importación de librerías

import tensorflow as tf
import numpy as np
import pandas as pd
import zipfile

from sklearn.model_selection import train_test_split

# Extracción de datos

#with zipfile.ZipFile("./datasets/train_ver2.csv.zip") as z:
#   with z.open("train_ver2.csv") as f:
#      dataset = pd.read_csv(f, nrows=10000)
#      print(dataset.head())    # print the first 5 rows

dataset=pd.read_csv('./datasets/train3.csv', low_memory=False)

In [2]:
#Vamos identificar las columnas a usar
dataset.head()

,fecha_dato,ncodpers,ind_empleado,pais_residencia,sexo,age,fecha_alta,es_nuevo_cliente,antiguedad,indrel,...,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
0,2015-01-28,1058935,N,ES,H,23,2012-09-12,0,34,1,...,0,0,0,0,0,0,0,0.0,0.0,0
1,2015-01-28,1054162,N,ES,H,44,2012-08-27,0,35,1,...,0,0,0,0,0,0,0,1.0,1.0,0
2,2015-01-28,1060757,N,ES,H,28,2012-09-14,0,32,1,...,0,0,0,0,0,0,0,0.0,0.0,0
3,2015-01-28,1029018,N,ES,V,23,2012-07-24,0,36,1,...,0,0,0,0,0,0,0,0.0,0.0,0
4,2015-01-28,1043474,N,ES,H,26,2012-08-06,0,35,1,...,0,0,0,0,0,0,0,0.0,0.0,0


In [3]:
dataset.columns[1:30]

Index(['ncodpers', 'ind_empleado', 'pais_residencia', 'sexo', 'age',
       'fecha_alta', 'es_nuevo_cliente', 'antiguedad', 'indrel',
       'ult_fec_cli_1t', 'indrel_1mes', 'tiprel_1mes', 'indresi', 'indext',
       'conyuemp', 'canal_entrada', 'indfall', 'tipodom', 'cod_prov',
       'nomprov', 'ind_actividad_cliente', 'renta', 'segmento', 'es_hombre',
       'es_cliente_activo', 'es_cliente_extranjero', 'canal_KHE', 'canal_KFC',
       'canal_KHD'],
      dtype='object')

Vamos a usar las siguientes variables: ID_PRODUCTO, COD_CLIENTE, y CANTIDAD

In [4]:

input_train=dataset[['age', 'antiguedad','ind_actividad_cliente','es_hombre', 'es_cliente_activo','es_cliente_extranjero',
               'provincia_MADRID','provincia_BARCELONA','provincia_PONTEVEDRA','provincia_SEVILLA','provincia_VALENCIA',
               'provincia_MALAGA','provincia_MURCIA','provincia_ZARAGOZA','provincia_CADIZ','provincia_BADAJOZ',
               'provincia_ALICANTE','provincia_CASTELLON','provincia_HUELVA','provincia_SALAMANCA','provincia_CORDOBA',
               'provincia_TOLEDO','provincia_CACERES','provincia_VALLADOLID','provincia_LERIDA','provincia_CANTABRIA',
               'provincia_OURENSE','provincia_LUGO','provincia_GIRONA','provincia_ALBACETE','provincia_CIUDAD_REAL','provincia_ASTURIAS',
               'provincia_BURGOS','provincia_GRANADA','provincia_TARRAGONA','provincia_CUENCA','provincia_ZAMORA','provincia_NAVARRA',
               'provincia_LEON','provincia_HUESCA','provincia_BIZKAIA','provincia_PALENCIA','provincia_JAEN','provincia_AVILA',
               'provincia_SEGOVIA','provincia_ALMERIA','provincia_SANTA_CRUZ_DE_TENERIFE','provincia_GUADALAJARA','provincia_TERUEL',
               'provincia_GIPUZKOA','provincia_SORIA','provincia_ALAVA','provincia_MELILLA','provincia_CEUTA','es_universitario',
               'es_particular','es_top','renta']]
output_train=dataset[['ind_ahor_fin_ult1', 'ind_aval_fin_ult1', 'ind_cco_fin_ult1',
       'ind_cder_fin_ult1', 'ind_cno_fin_ult1', 'ind_ctju_fin_ult1',
       'ind_ctma_fin_ult1', 'ind_ctop_fin_ult1', 'ind_ctpp_fin_ult1',
       'ind_deco_fin_ult1', 'ind_deme_fin_ult1', 'ind_dela_fin_ult1',
       'ind_ecue_fin_ult1', 'ind_fond_fin_ult1', 'ind_hip_fin_ult1',
       'ind_plan_fin_ult1', 'ind_pres_fin_ult1', 'ind_reca_fin_ult1',
       'ind_tjcr_fin_ult1', 'ind_valo_fin_ult1', 'ind_viv_fin_ult1',
       'ind_nomina_ult1', 'ind_nom_pens_ult1', 'ind_recibo_ult1' ]]

In [5]:
input_train.shape

(16482, 58)

In [6]:
nodos_hidden=250 # Cantidad de nodos de la capa oculta
x=tf.placeholder(tf.float32, [None, 58],name='Inputs') # Es la matriz de inputs. Actualmetne solo hay seis
y_=tf.placeholder(tf.float32, [None, 24], name='Outputs')  # 24 productos financieros a recomendar
# Matriz de pesos a encontrar. Esta es una variable y por eso no puede ser del tipo
## tf.placeholder, sino que tiene que ser tipo variable para que se entienda que esta es la variable a optimizar
W0=tf.Variable(tf.random_normal([58,nodos_hidden], stddev=0.01), name='Capa_Oculta1') 
B0= tf.Variable(tf.random_normal([1,nodos_hidden],stddev=0.01), name='sesgo_Capa_Oculta1') # Está es la constante o sesgo del perceptron

W1=tf.Variable(tf.random_normal([nodos_hidden,24],stddev=0.01),name='Capa_Oculta2') ## Esta es la matriz de pesos de la segunda capa
B1= tf.Variable(tf.random_normal([1,24],stddev=0.01), name='Sesgo_capa_oculta2') # Está es la constante o sesgo del perceptron

In [7]:
# Vamos ahora a conectarla

capa_oculta=tf.nn.relu(tf.matmul(x,W0)+B0)
y=tf.matmul(capa_oculta,W1)+B1

In [8]:
# Calculamos la función de costo 
cross_entropy= tf.nn.sigmoid_cross_entropy_with_logits(labels=y_,logits=y)

In [9]:
# Ahora Vamos a crear la funcion a optimizar
optimizador= tf.train.AdamOptimizer(0.005).minimize(cross_entropy)

In [10]:
init=tf.global_variables_initializer() 
session= tf.Session()
session.run(init)
writer = tf.summary.FileWriter('./hackaton_tensorboard', session.graph)


In [11]:
input_train.shape


(16482, 58)

In [13]:
train_error=[]
test_error=[]

for i in range(150):
    trainX,testX, trainY,testY= train_test_split(input_train, output_train, random_state=42)
    session.run(optimizador,feed_dict={x:trainX, y_:trainY}) #feed_dict es un diccionario con los datos de entrenamiento
    loss_train=session.run(cross_entropy,feed_dict={x:trainX, y_:trainY})
    loss_test=session.run(cross_entropy,feed_dict={x:testX, y_:testY})
    train_error.append(loss_train) # calcula el error sobre datos de training
    test_error.append(loss_test) #calcula error en los datos de testing
    
print('Terminó el entrenamiento con Gradiente descendente Estocástico')

Terminó el entrenamiento con Gradiente descendente Estocástico


In [41]:
trainX,trainY, TestX,TestY= train_test_split(input_train, output_train, random_state=42)

trainY.shape

(4121, 73)